**Assumptions**
- A invocation run does not require context from a previous run. No chat history -> No tweaks/edits
- We are only considering NSE for live data

**Cheat Sheet for Qualitative Analysis**

| Metric / Insight     | Signal To Look For                        |
| -------------------- | ----------------------------------------- |
| Total Revenue        | Growth trend, seasonal strength/weakness  |
| Operating Margin     | Cost control or inflationary pressure     |
| Net Profit / EPS     | Health of the business, investor returns  |
| Recurring Themes     | Strategic focus areas or persistent risks |
| Management Sentiment | Confidence vs. hedging                    |
| Forward Guidance     | Directional clues (explicit or subtle)    |
| Segment Performance  | What’s winning or underperforming         |


## Observations:
    1. YFinance tools + Simple ReAct Agent -> [2/5] (High token usage, incomplete data points)

In [3]:
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv
import os

load_dotenv()


llm = init_chat_model(
    "gemini-2.0-flash",
    model_provider="google_genai",
    temperature=0.2,
    api_key=os.getenv("GOOGLE_API_KEY"),
)

In [ ]:
# from langchain_mcp_adapters.client import MultiServerMCPClient

# client = MultiServerMCPClient(
#     {
#         "finance": {
#             "url": "https://yfinance-mcp-owvg.onrender.com/mcp/",
#             "transport": "streamable_http",
#         }
#     }
# )

# tools = await client.get_tools()


In [5]:
from langgraph.prebuilt import create_react_agent
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

collection_name = "transcripts"

client = QdrantClient(url="http://localhost:6333")
vector_store = QdrantVectorStore(
    client=client,
    collection_name=collection_name,
    embedding=embeddings,
)


def search_docs(topics_list: list[str]):
    """Search for a list of metrics/values/figures or general information in the company X's earnings call data."""
    results = []
    for query in topics_list:
        result = vector_store.similarity_search(query, k=3)
        results.append(
            {
                "query": query,
                "top_results": [results.page_content for results in result],
            }
        )
    return results


agent = create_react_agent(
    model=llm,
    tools=[search_docs],
    debug=True,
    prompt="You are an expert financial analyst. Use the provided tools to research for a task, and ALWAYS take the step-by-step approach. Reflect after each step to decide whether you have everything you need.",
)

In [6]:
await agent.ainvoke(
    {
        "messages": {
            "role": "user",
            "content": "Analyze the financial reports for ticker = 'TCS.NS' for the past quarters and provide a qualitative forecast for the upcoming quarter. Your forecast must identify key financial trends (e.g., revenue growth, margin pressure), summarize management's stated outlook, and highlight any significant risks or opportunities mentioned.",
        }
    }
)

[values] {'messages': [HumanMessage(content="Analyze the financial reports for ticker = 'TCS.NS' for the past quarters and provide a qualitative forecast for the upcoming quarter. Your forecast must identify key financial trends (e.g., revenue growth, margin pressure), summarize management's stated outlook, and highlight any significant risks or opportunities mentioned.", additional_kwargs={}, response_metadata={}, id='01c334f9-2caa-44d9-87ac-4a4c65738cc5')]}
[updates] {'agent': {'messages': [AIMessage(content="Okay, I can analyze the financial reports for 'TCS.NS' and provide a qualitative forecast. Here's my plan:\n\n1.  **Identify Key Financial Trends:** I'll use the `search_docs` tool to look for information on revenue growth, margins, and other relevant financial metrics.\n2.  **Summarize Management's Outlook:** I'll use the `search_docs` tool to find statements from management regarding their outlook for the next quarter.\n3.  **Highlight Risks and Opportunities:** I'll use the `

{'messages': [HumanMessage(content="Analyze the financial reports for ticker = 'TCS.NS' for the past quarters and provide a qualitative forecast for the upcoming quarter. Your forecast must identify key financial trends (e.g., revenue growth, margin pressure), summarize management's stated outlook, and highlight any significant risks or opportunities mentioned.", additional_kwargs={}, response_metadata={}, id='01c334f9-2caa-44d9-87ac-4a4c65738cc5'),
  AIMessage(content="Okay, I can analyze the financial reports for 'TCS.NS' and provide a qualitative forecast. Here's my plan:\n\n1.  **Identify Key Financial Trends:** I'll use the `search_docs` tool to look for information on revenue growth, margins, and other relevant financial metrics.\n2.  **Summarize Management's Outlook:** I'll use the `search_docs` tool to find statements from management regarding their outlook for the next quarter.\n3.  **Highlight Risks and Opportunities:** I'll use the `search_docs` tool to identify any risks or

In [ ]:
# agent.invoke(
#     {
#         "messages": {
#             "role": "user",
#             "content": "Analyze the financial reports and transcripts for the past quarters and provide a qualitative forecast for the upcoming quarter. Your forecast must identify key financial trends (e.g., revenue growth, margin pressure), summarize management's stated outlook, and highlight any significant risks or opportunities mentioned.",
#         }
#     }
# )

# TODO: A robust tool designed to understand quarterly financial reports and extract key financial metrics